In [1]:
# Prerequesites: install "intake-esm", "requests", "aiohttp", "gcsfs"
import intake
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Load CMIP6 via intake-esm
col_url = "https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml"
# col = intake.open_esm_datastore(col_url)
col = intake.open_catalog(col_url)
# Search catalog
query = col.search(
    experiment_id="historical",
    table_id="SImon",
    variable_id=["siu", "siv", "siconc"],
)

TypeError: search() got an unexpected keyword argument 'experiment_id'

In [4]:
col.search("cmip6")

master_search:
  args:
    getenv: true
    getshell: true
    name: master_search
    storage_options: null
    ttl: 60
    user_parameters: {}
  description: ''
  driver: intake.catalog.base.Catalog
  metadata: {}


In [3]:
query.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,NOAA-GFDL,GFDL-ESM4,historical,r3i1p1f1,SImon,siconc,gn,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/hist...,NaN,20180701
1,CMIP,NOAA-GFDL,GFDL-ESM4,historical,r3i1p1f1,SImon,siconc,gr,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/hist...,NaN,20180701
2,CMIP,NOAA-GFDL,GFDL-ESM4,historical,r2i1p1f1,SImon,siconc,gn,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/hist...,NaN,20180701
3,CMIP,NOAA-GFDL,GFDL-ESM4,historical,r2i1p1f1,SImon,siconc,gr,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/hist...,NaN,20180701
4,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,SImon,siu,gn,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/histo...,NaN,20180701
...,...,...,...,...,...,...,...,...,...,...,...
1130,CMIP,MRI,MRI-ESM2-0,historical,r6i1p1f1,SImon,siconc,gn,gs://cmip6/CMIP6/CMIP/MRI/MRI-ESM2-0/historica...,NaN,20210909
1131,CMIP,MRI,MRI-ESM2-0,historical,r7i1p1f1,SImon,siconc,gn,gs://cmip6/CMIP6/CMIP/MRI/MRI-ESM2-0/historica...,NaN,20210909
1132,CMIP,MRI,MRI-ESM2-0,historical,r8i1p1f1,SImon,siconc,gn,gs://cmip6/CMIP6/CMIP/MRI/MRI-ESM2-0/historica...,NaN,20210909
1133,CMIP,MRI,MRI-ESM2-0,historical,r9i1p1f1,SImon,siconc,gn,gs://cmip6/CMIP6/CMIP/MRI/MRI-ESM2-0/historica...,NaN,20210909


In [7]:
institutions = pd.DataFrame(
    data=[(name, len(group), len(np.unique(group['source_id'])), len(np.unique(group['member_id']))) for name, group in query.df.groupby('institution_id')],
    columns=['institution_id', 'number_of_datasets', 'number_of_models', 'number_of_ensembles'])
institutions.set_index('institution_id', inplace=True)
institutions.index

Index(['AS-RCEC', 'AWI', 'BCC', 'CAMS', 'CAS', 'CCCma', 'CMCC', 'CNRM-CERFACS',
       'CSIRO', 'CSIRO-ARCCSS', 'E3SM-Project', 'EC-Earth-Consortium',
       'FIO-QLNM', 'HAMMOZ-Consortium', 'INM', 'IPSL', 'KIOST', 'MIROC',
       'MOHC', 'MPI-M', 'MRI', 'NASA-GISS', 'NCAR', 'NCC', 'NIMS-KMA',
       'NOAA-GFDL', 'NUIST', 'SNU', 'THU'],
      dtype='object', name='institution_id')

In [14]:
model_query = col.search(
        source_id='CAS-ESM2-0',
        experiment_id="historical",
        table_id="SImon",
        grid_label="gn",
        variable_id=["siu", "siv", "siconc"],
    )

In [18]:
results = model_query.to_dataset_dict(
    zarr_kwargs={'consolidated': True},
    chunks={'time': 12, 'x': 180, 'y': 180},
    xarray_combine_by_coords_kwargs={'compat': 'no_conflicts'}
)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [25]:
ensemble_members = results['CMIP.CAS.CAS-ESM2-0.historical.SImon.gn'].member_id.data

In [27]:
for ens in ensemble_members:
    ds = results['CMIP.CAS.CAS-ESM2-0.historical.SImon.gn'].sel(member_id=ens)
    

In [10]:
data = {}
for inst_id in ['NCAR', 'NASA-GISS', 'NOAA-GFDL']:
    inst_query = col.search(
        institution_id=inst_id,
        experiment_id="historical",
        table_id="SImon",
        variable_id=["siu", "siv", "siconc"],
    )
    try:
        data[inst_id] = inst_query.to_dataset_dict(
            zarr_kwargs={'consolidated': True},
            chunks={'time': 12, 'x': 180, 'y': 180},
            xarray_combine_by_coords_kwargs={'compat': 'no_conflicts'},
            skip_on_error=True)
    except:
        print(inst_id)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


/opt/miniconda3/envs/sea_ice_env/lib/python3.9/site-packages/dask/array/core.py:4849: PerformanceWarning: Increasing number of chunks by factor of 11
  result = blockwise(
/opt/miniconda3/envs/sea_ice_env/lib/python3.9/site-packages/dask/array/core.py:4849: PerformanceWarning: Increasing number of chunks by factor of 11
  result = blockwise(



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [13]:
data['NASA-GISS']

{'CMIP.NASA-GISS.GISS-E2-2-H.historical.SImon.gr': <xarray.Dataset>
 Dimensions:         (lat: 180, bnds: 2, lon: 360, member_id: 5,
                      dcpp_init_year: 1, time: 1980)
 Coordinates:
   * lat             (lat) float64 -90.0 -88.5 -87.5 -86.5 ... 87.5 88.5 90.0
     lat_bnds        (lat, bnds) float64 dask.array<chunksize=(180, 2), meta=np.ndarray>
   * lon             (lon) float64 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
     lon_bnds        (lon, bnds) float64 dask.array<chunksize=(360, 2), meta=np.ndarray>
   * time            (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
     time_bnds       (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
     type            |S7 b'sea_ice'
   * member_id       (member_id) object 'r1i1p1f1' 'r2i1p1f1' ... 'r5i1p1f1'
   * dcpp_init_year  (dcpp_init_year) float64 nan
 Dimensions without coordinates: bnds
 Data variables:
     siconc          (member_id, dcpp_init_year, time, lat, lon) float32 dask.

In [2]:
# #for ds_name, ds in dset_dict.items():
# print(f"Dataset: {ds_name}")
# print(ds.data_vars)

# Access first dataset from search
# for model in dset_dict:
#    ds = dset_dict[model]
#    check if all variables in ds. if so, add model to list of accepted ones
model = 'CMIP.IPSL.IPSL-CM6A-LR.historical.SImon.gn'
ds = dset_dict[model]

In [9]:
model = 'CMIP.NCAR.CESM2.historical.SImon.gn'
ds = dset_dict[model]


In [4]:
dset_dict.keys()


dict_keys(['CMIP.IPSL.IPSL-CM6A-LR-INCA.historical.SImon.gn', 'CMIP.NCC.NorCPM1.historical.SImon.gn', 'CMIP.AWI.AWI-CM-1-1-MR.historical.SImon.gn', 'CMIP.AS-RCEC.TaiESM1.historical.SImon.gn', 'CMIP.IPSL.IPSL-CM5A2-INCA.historical.SImon.gn', 'CMIP.NCC.NorESM2-MM.historical.SImon.gn', 'CMIP.CMCC.CMCC-CM2-SR5.historical.SImon.gn', 'CMIP.CAS.FGOALS-g3.historical.SImon.gn', 'CMIP.THU.CIESM.historical.SImon.gn', 'CMIP.MPI-M.ICON-ESM-LR.historical.SImon.gn', 'CMIP.MPI-M.MPI-ESM1-2-LR.historical.SImon.gn', 'CMIP.MRI.MRI-ESM2-0.historical.SImon.gn', 'CMIP.CMCC.CMCC-ESM2.historical.SImon.gn', 'CMIP.NCAR.CESM2-WACCM.historical.SImon.gn', 'CMIP.HAMMOZ-Consortium.MPI-ESM-1-2-HAM.historical.SImon.gn', 'CMIP.EC-Earth-Consortium.EC-Earth3-Veg-LR.historical.SImon.gn', 'CMIP.MPI-M.MPI-ESM1-2-HR.historical.SImon.gn', 'CMIP.CAMS.CAMS-CSM1-0.historical.SImon.gn', 'CMIP.NOAA-GFDL.GFDL-ESM4.historical.SImon.gn', 'CMIP.CCCma.CanESM5.historical.SImon.gn', 'CMIP.CAS.CAS-ESM2-0.historical.SImon.gn', 'CMIP.EC-Ear

In [14]:
ds.attrs.keys()

dict_keys(['Conventions', 'activity_id', 'branch_method', 'branch_time_in_child', 'branch_time_in_parent', 'case_id', 'cesm_casename', 'contact', 'experiment', 'experiment_id', 'external_variables', 'forcing_index', 'frequency', 'further_info_url', 'grid', 'grid_label', 'initialization_index', 'institution', 'institution_id', 'license', 'mip_era', 'model_doi_url', 'nominal_resolution', 'parent_activity_id', 'parent_experiment_id', 'parent_mip_era', 'parent_source_id', 'parent_time_units', 'parent_variant_label', 'physics_index', 'product', 'realization_index', 'realm', 'source', 'source_id', 'source_type', 'sub_experiment', 'sub_experiment_id', 'table_id', 'variant_info', 'variant_label', 'intake_esm_attrs:activity_id', 'intake_esm_attrs:institution_id', 'intake_esm_attrs:source_id', 'intake_esm_attrs:experiment_id', 'intake_esm_attrs:member_id', 'intake_esm_attrs:table_id', 'intake_esm_attrs:grid_label', 'intake_esm_attrs:_data_format_', 'intake_esm_dataset_key'])

In [3]:
import pandas as pd
metadata = []
for model in dset_dict:
    metadata.append([model, dset_dict[model].attrs['institution'], dset_dict[model].attrs['nominal_resolution']])
metadata_table = pd.DataFrame(metadata, columns=['model', 'institution', 'resolution'])
metadata_table.to_csv('metadata.csv')

In [6]:
ds = dset_dict['CMIP.AS-RCEC.TaiESM1.historical.SImon.gn']
ds

<xarray.Dataset>
Dimensions:             (i: 320, j: 384, member_id: 1, dcpp_init_year: 1,
                         time: 1980, bnds: 2, vertices: 4)
Coordinates:
  * i                   (i) int32 1 2 3 4 5 6 7 ... 314 315 316 317 318 319 320
  * j                   (j) int32 1 2 3 4 5 6 7 ... 378 379 380 381 382 383 384
    latitude            (j, i) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    longitude           (j, i) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
  * time                (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:0...
    time_bnds           (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
    type                |S7 ...
  * member_id           (member_id) object 'r1i1p1f1'
  * dcpp_init_year      (dcpp_init_year) float64 nan
Dimensions without coordinates: bnds, vertices
Data variables:
    siconc              (member_id, dcpp_init_year, time, j, i) float32 dask.array<chunksize=(1, 1, 738, 384, 320), meta=np.ndarray>
    vertices_latitude   (j, i, vertices) float64 dask.array<chunksize=(384, 320, 4), meta=np.ndarray>
    vertices_longitude  (j, i, vertices) float64 dask.array<chunksize=(384, 320, 4), meta=np.ndarray>
Attributes: (12/66)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      CMIP
    branch_method:                    Hybrid-restart from year 0671-01-01 of ...
    branch_time:                      0.0
    branch_time_in_child:             0.0
    branch_time_in_parent:            171550.0
    ...                               ...
    intake_esm_attrs:variable_id:     siconc
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:zstore:          gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/h...
    intake_esm_attrs:version:         20200630
    intake_esm_attrs:_data_format_:   zarr
    intake_esm_dataset_key:           CMIP.AS-RCEC.TaiESM1.historical.SImon.gn

In [14]:
', '.join([x for x in ds.coords])

'i, j, latitude, longitude, time, time_bnds, type, member_id, dcpp_init_year'

In [4]:
# time filter
time = ds['time'].compute()
time_mask = time >= np.datetime64('2000-01-01')
ds_filtered_time = ds.where(time_mask, drop=True)

# latitude filter
lat = ds_filtered_time['y'].compute()
lat_mask = lat >= 75
ds_north_75 = ds_filtered_time.where(lat_mask, drop=True)

# Filter out land, open and zero velocities
condition = (
    (ds_north_75['siconc'] > 0) &  
    (ds_north_75['siu'] != 0) &   
    (ds_north_75['siv'] != 0)    
)

condition = condition.compute()

ds_filtered = ds_north_75.where(condition, drop=True)

# magnitude of sea ice velocity vector
velocity_magnitude = np.sqrt(ds_filtered['siu']**2 + ds_filtered['siv']**2)

# area-averaged sea ice velocity
# Check if 'area' variable exists; if not, assume equal weighting
if 'area' in ds_filtered:
    area = ds_filtered['area']
else:
    # If 'area' is not available, assume equal weighting
    area = xr.ones_like(velocity_magnitude)

area_weighted_velocity = (
    (velocity_magnitude * area).sum(dim=['y', 'x']) /
    area.sum(dim=['y', 'x'])
)

area_weighted_velocity.plot()
plt.title('Area-Averaged Sea Ice Velocity North of 75° Latitude')
plt.ylabel('Sea Ice Velocity (m/s)')
plt.xlabel('Time')
plt.show()


NameError: name 'ds' is not defined